In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string 
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium
import json
import vincent
import branca.colormap as cm
import scipy.spatial as spatial

In [2]:
London_LSOAs = gpd.read_file('map_data/LSOA/London_LSOA_shape_cleaned.shp')
Buildings = gpd.read_file('map_data/OSM/gis.osm_buildings_a_free_1.shp')
Landuse = gpd.read_file('map_data/OSM/gis.osm_landuse_a_free_1.shp')

In [3]:
Buildings['lon'] = Buildings.geometry.centroid.x
Buildings['lat'] = Buildings.geometry.centroid.y

London_LSOAs['lon'] = London_LSOAs.geometry.centroid.x
London_LSOAs['lat'] = London_LSOAs.geometry.centroid.y

In [4]:
#Convert LSOA locations into a KDTree
LSOA_tree = spatial.cKDTree(London_LSOAs[['lon', 'lat']].as_matrix())

In [5]:
Building_point_strings = Buildings['lat'].map(str) + ',' + Buildings['lon'].map(str)
Buildings['point_strings'] = Building_point_strings

In [6]:
def find_LSOA_from_point(point, approximate):
    (lat, lon) = [float(i) for i in point.split(',')]
    point = Point((lon, lat))
    nearest_n_polygons = London_LSOAs.loc[LSOA_tree.query((lon, lat), 5)[1]]
    #Set landuse_id to the closest polygon by default
    building_LSOA_id = -999
    if approximate == True:
        building_LSOA_id = nearest_n_polygons.lsoa11cd.values[0]
    #Check if point is in the nearest_n, if yes, replace (reversed rows so closest shape is reviewed last)
    for row in nearest_n_polygons.iloc[::-1].iterrows():
        if point.within(row[1].geometry):
            building_LSOA_id = row[1].lsoa11cd
    return building_LSOA_id

LSOA_precise = []

for index, string in enumerate(Building_point_strings):
    if index % 100000 == 0:
        print index
    LSOA_precise.append(find_LSOA_from_point(string, approximate = True))

0
100000
200000
300000


In [7]:
Building_LSOA_dict = pd.Series(pd.Series(LSOA_precise).values,index=Building_point_strings).to_dict()
Buildings['LSOA'] = Buildings.point_strings.map(Building_LSOA_dict)

In [8]:
LSOA_name_dict = pd.Series(London_LSOAs.District.values,index=London_LSOAs.lsoa11cd).to_dict()
Buildings['District'] = Buildings.LSOA.map(LSOA_name_dict)

In [9]:
Buildings = Buildings.drop('point_strings', axis = 1)
#Save down shape file
Buildings.to_file(filename= 'map_data/Buildings_and_LSOA.shp', driver='GeoJSON', encoding='utf-8') 

In [10]:
Camden = Buildings[Buildings.District == 'Camden']
Hackney = Buildings[Buildings.District == 'Hackney']

choropleth_json = gpd.GeoDataFrame(Hackney).to_json()
Choro_dict = Hackney[['osm_id', 'code']].set_index('osm_id')['code']

linear = cm.LinearColormap(['red','black','blue'], 
                           vmin=Hackney['code'].quantile(.1),
                           vmax=Hackney['code'].quantile(.9))

linear.caption = 'code'

choropleths = folium.GeoJson(
        json.loads(choropleth_json),
        smooth_factor = 1.0,
        style_function=lambda feature: 
        {
        'fillColor': linear(Choro_dict[feature['properties']['osm_id']]),
        'color' : 'black',
        'weight' : 1,
        'dashArray' : '5, 5'
        },
        )

m = folium.Map(location=[51.507, -0.128], tiles='Stamen Toner', zoom_start=12)

m.add_child(linear)
m.add_child(choropleths)

m.save('visualisations/Hackney_Buildings.html')